In [4]:
import sys
import os
import pdb
import torch
import argparse

this_filepath = os.path.abspath('')
megatron_root = os.path.abspath(
    this_filepath + "../../..")
print(f"megatron_root: {megatron_root}")

sys.path.insert(0, megatron_root)

import copy
import subprocess
import tempfile
import torch
from transformers import AutoConfig, AutoModel, AutoModelForCausalLM

from megatron.arguments import parse_args
from megatron.global_vars import set_args
from tests.unit_tests.test_utilities import Utils
from megatron.core.tensor_parallel import model_parallel_cuda_manual_seed
from megatron.core.parallel_state import _set_global_memory_buffer

megatron_root: /cpfs/29ccba8f16c61395/data/user/liushan/projects/demo/Megatron-LM


/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Zarr-based strategies will not be registered because of missing packages


In [3]:
!pwd
# Download the huggingface model from oss.
!rclone copy --transfers 16 --checkers 16 oss://inf-alpha/home/megatron/tests/test_convert_ckpt/models/Llama-2-13b-hf ./models/Llama-2-13b-hf
llama_root = "./models/Llama-2-13b-hf"
!ls {llama_root}

/cpfs/29ccba8f16c61395/data/user/liushan/projects/demo/Megatron-LM/tests/unit_tests
config.json			  modeling_llama.py
configuration_llama.py		  special_tokens_map.json
model-00001-of-00003.safetensors  tokenizer.json
model-00002-of-00003.safetensors  tokenizer.model
model-00003-of-00003.safetensors  tokenizer_config.json
model.safetensors.index.json


In [6]:
# Show current git revision of Megatron-LM (expected: cb5a40e16ef163cfb65face3dcbfea8fa624425a)
!git rev-parse HEAD

cb5a40e16ef163cfb65face3dcbfea8fa624425a


## Convert Llama-2-13b-hf to Megatron checkpoint.

In [7]:
# import unicorn
sys.path.append(os.path.join(megatron_root, "tools", "unicorn"))
import unicorn

[2023-11-14 08:55:40,316] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [8]:
def _parse_args():
    parser = argparse.ArgumentParser()
    parser = unicorn.add_checkpointing_args(parser)
    parser = unicorn.add_transformers_checkpoint_args(parser)
    parser = unicorn.add_megatron_checkpoint_args(parser)
    args = parser.parse_args()
    return args


sys.argv = ['script.py',
            '--megatron-path', megatron_root,
            '--load-path', llama_root,
            '--save-path', os.path.join(this_filepath, "models", "llama-megatron-t2p2"),
            '--model-name', 'llama2-13b',
            '--template-name', 'llama',
            '--print-checkpoint-structure',
            '--target_tensor_model_parallel_size', '2',
            '--target_pipeline_model_parallel_size', '2',
            '--target_params_dtype', 'fp16']

args = _parse_args()
print(args)

Namespace(megatron_path='/cpfs/29ccba8f16c61395/data/user/liushan/projects/demo/Megatron-LM', convert_checkpoint_from_megatron_to_transformers=False, load_path='./models/Llama-2-13b-hf', save_path='/cpfs/29ccba8f16c61395/data/user/liushan/projects/demo/Megatron-LM/tests/unit_tests/models/llama-megatron-t2p2', model_name='llama2-13b', template_name='llama', print_checkpoint_structure=True, tokenizer_name=None, max_shard_size='10GB', target_tensor_model_parallel_size=2, target_pipeline_model_parallel_size=2, target_data_parallel_size=1, target_params_dtype='fp16', make_vocab_size_divisible_by=128, extra_num_vocabs=0, use_distributed_optimizer=False, iteration=0)


In [9]:
# make tp=2&pp=2 megatron checkpoint
unicorn.convert_checkpoint_from_transformers_to_megatron(args)

=> Loading ./models/Llama-2-13b-hf/model-00001-of-00003.safetensors ...
=> Loading ./models/Llama-2-13b-hf/model-00003-of-00003.safetensors ...
=> Loading ./models/Llama-2-13b-hf/model-00002-of-00003.safetensors ...


cp: not writing through dangling symlink '/cpfs/29ccba8f16c61395/data/user/liushan/projects/demo/Megatron-LM/tests/unit_tests/models/llama-megatron-t2p2/config.json'
cp: not writing through dangling symlink '/cpfs/29ccba8f16c61395/data/user/liushan/projects/demo/Megatron-LM/tests/unit_tests/models/llama-megatron-t2p2/model.safetensors.index.json'
cp: not writing through dangling symlink '/cpfs/29ccba8f16c61395/data/user/liushan/projects/demo/Megatron-LM/tests/unit_tests/models/llama-megatron-t2p2/special_tokens_map.json'
cp: not writing through dangling symlink '/cpfs/29ccba8f16c61395/data/user/liushan/projects/demo/Megatron-LM/tests/unit_tests/models/llama-megatron-t2p2/tokenizer.json'
cp: not writing through dangling symlink '/cpfs/29ccba8f16c61395/data/user/liushan/projects/demo/Megatron-LM/tests/unit_tests/models/llama-megatron-t2p2/tokenizer_config.json'
cp: cannot stat './models/Llama-2-13b-hf/*.tiktoken': No such file or directory


=> Converting ...
=> converting embedding layer ...
=> Converting transformer blocks ...
Checkpoint structure of model state dict shard belonging to TP rank 0 and PP rank 0:
# model                                           
..# language_model                                
....# embedding                                   
......# word_embeddings                           
........# weight                                   : torch.Size([16000, 5120])
....# output_layer                                
......# weight                                     : torch.Size([16000, 5120])
....# encoder                                     
......# layers.0.input_norm.weight                 : torch.Size([5120])
......# layers.0.self_attention.query_key_value.weight : torch.Size([7680, 5120])
......# layers.0.self_attention.dense.weight       : torch.Size([5120, 2560])
......# layers.0.post_attention_norm.weight        : torch.Size([5120])
......# layers.0.mlp.dense_h_to_4h.weight          : torch

In [10]:
# make tp=1&pp=1 megatron checkpoint
args.save_path = os.path.join(this_filepath, "models", "llama-megatron-t1p1")
args.target_tensor_model_parallel_size = 1
args.target_pipeline_model_parallel_size = 1

unicorn.convert_checkpoint_from_transformers_to_megatron(args)

=> Loading ./models/Llama-2-13b-hf/model-00001-of-00003.safetensors ...
=> Loading ./models/Llama-2-13b-hf/model-00003-of-00003.safetensors ...
=> Loading ./models/Llama-2-13b-hf/model-00002-of-00003.safetensors ...


cp: not writing through dangling symlink '/cpfs/29ccba8f16c61395/data/user/liushan/projects/demo/Megatron-LM/tests/unit_tests/models/llama-megatron-t1p1/config.json'
cp: not writing through dangling symlink '/cpfs/29ccba8f16c61395/data/user/liushan/projects/demo/Megatron-LM/tests/unit_tests/models/llama-megatron-t1p1/model.safetensors.index.json'
cp: not writing through dangling symlink '/cpfs/29ccba8f16c61395/data/user/liushan/projects/demo/Megatron-LM/tests/unit_tests/models/llama-megatron-t1p1/special_tokens_map.json'
cp: not writing through dangling symlink '/cpfs/29ccba8f16c61395/data/user/liushan/projects/demo/Megatron-LM/tests/unit_tests/models/llama-megatron-t1p1/tokenizer.json'
cp: not writing through dangling symlink '/cpfs/29ccba8f16c61395/data/user/liushan/projects/demo/Megatron-LM/tests/unit_tests/models/llama-megatron-t1p1/tokenizer_config.json'
cp: cannot stat './models/Llama-2-13b-hf/*.tiktoken': No such file or directory


=> Converting ...
=> converting embedding layer ...
=> Converting transformer blocks ...
Checkpoint structure of model state dict shard belonging to TP rank 0 and PP rank 0:
# model                                           
..# language_model                                
....# embedding                                   
......# word_embeddings                           
........# weight                                   : torch.Size([32000, 5120])
....# output_layer                                
......# weight                                     : torch.Size([32000, 5120])
....# encoder                                     
......# layers.0.input_norm.weight                 : torch.Size([5120])
......# layers.0.self_attention.query_key_value.weight : torch.Size([15360, 5120])
......# layers.0.self_attention.dense.weight       : torch.Size([5120, 5120])
......# layers.0.post_attention_norm.weight        : torch.Size([5120])
......# layers.0.mlp.dense_h_to_4h.weight          : torc

In [11]:
# Use tools/checkpoint in Megatron-LM to convert llama-megatron-t2p2 to megatron-util-t1p1
cmds = [
    "python",
    "tools/checkpoint/util.py",
    "--model-type", "GPT",
    "--load-dir", os.path.join(this_filepath, "models", "llama-megatron-t2p2"),
    "--save-dir", os.path.join(this_filepath, "models", "megatron-util-t1p1"),
    "--target-tensor-parallel-size", "1",
    "--target-pipeline-parallel-size", "1"
]

subprocess.run(cmds, cwd=megatron_root, env={"PYTHONPATH":megatron_root})

Loaded loader_megatron as the loader.
Loaded saver_megatron as the saver.
Starting saver...


Zarr-based strategies will not be registered because of missing packages
Zarr-based strategies will not be registered because of missing packages


Starting loader...
Setting num_layers to 40 from checkpoint
Setting hidden_size to 5120 from checkpoint
Setting ffn_hidden_size to 13824 from checkpoint
Setting seq_length to 4096 from checkpoint
Setting num_attention_heads to 40 from checkpoint
Checkpoint did not provide arguments num_query_groups
Setting kv_channels to 128 from checkpoint
Setting max_position_embeddings to 4096 from checkpoint
Setting position_embedding_type to rope from checkpoint
Checkpoint did not provide arguments rotary_percent
Setting add_bias_linear to False from checkpoint
Setting swiglu to True from checkpoint
Setting untie_embeddings_and_output_weights to True from checkpoint
Checkpoint did not provide arguments apply_layernorm_1p
Setting normalization to RMSNorm from checkpoint
Setting tokenizer_type to NullTokenizer from checkpoint
Setting padded_vocab_size to 32000 from checkpoint
Setting tensor_model_parallel_size to 2 from checkpoint
Setting pipeline_model_parallel_size to 2 from checkpoint
Checkpoint 

CompletedProcess(args=['python', 'tools/checkpoint/util.py', '--model-type', 'GPT', '--load-dir', '/cpfs/29ccba8f16c61395/data/user/liushan/projects/demo/Megatron-LM/tests/unit_tests/models/llama-megatron-t2p2', '--save-dir', '/cpfs/29ccba8f16c61395/data/user/liushan/projects/demo/Megatron-LM/tests/unit_tests/models/megatron-util-t1p1', '--target-tensor-parallel-size', '1', '--target-pipeline-parallel-size', '1'], returncode=0)

In [12]:
# Compare the converted checkpoints
# src: converted by unicorn
# dst: converted by Megatron tools/checkpoint/util
src_ckpt = os.path.join(this_filepath, "models", "llama-megatron-t1p1", "release", "mp_rank_00", "model_optim_rng.pt")
dst_ckpt = os.path.join(this_filepath, "models", "megatron-util-t1p1", "iter_0000000", "mp_rank_00", "model_optim_rng.pt")

unicorn_state_dict = torch.load(src_ckpt, map_location="cpu")
util_state_dict = torch.load(dst_ckpt, map_location="cpu")

In [13]:
unicorn_model = unicorn_state_dict['model']['language_model']
util_model = util_state_dict['model']['language_model']

# Compare word_embedding
unicorn_wte = unicorn_model['embedding']['word_embeddings']['weight']
util_wte = util_model['embedding']['word_embeddings']['weight']
wte_diff = (unicorn_wte != util_wte).sum()
print(f"wte diff: {wte_diff}")

# Compare lm_head
unicorn_lm_head = unicorn_model['output_layer']['weight']
util_lm_head = util_model['output_layer']['weight']
lm_head_diff = (unicorn_lm_head != util_lm_head).sum()
print(f"lm_head diff: {lm_head_diff}")

# Compare transformer block
diff_records = {}
for k in util_model['encoder'].keys():
    unicorn_v = unicorn_model['encoder'][k]
    util_v = util_model['encoder'][k]
    diff_v = (unicorn_v != util_v).sum()
    print(f"{k} diff: {diff_v}")
    if diff_v > 0:
        diff_records[k] = diff_v
print(f"ALL diffs: {diff_records}")

wte diff: 0
lm_head diff: 0
layers.0.input_norm.weight diff: 0
layers.0.self_attention.query_key_value.weight diff: 0
layers.0.self_attention.dense.weight diff: 0
layers.0.post_attention_norm.weight diff: 0
layers.0.mlp.dense_h_to_4h.weight diff: 0
layers.0.mlp.dense_4h_to_h.weight diff: 0
layers.1.input_norm.weight diff: 0
layers.1.self_attention.query_key_value.weight diff: 0
layers.1.self_attention.dense.weight diff: 0
layers.1.post_attention_norm.weight diff: 0
layers.1.mlp.dense_h_to_4h.weight diff: 0
layers.1.mlp.dense_4h_to_h.weight diff: 0
layers.2.input_norm.weight diff: 0
layers.2.self_attention.query_key_value.weight diff: 0
layers.2.self_attention.dense.weight diff: 0
layers.2.post_attention_norm.weight diff: 0
layers.2.mlp.dense_h_to_4h.weight diff: 0
layers.2.mlp.dense_4h_to_h.weight diff: 0
layers.3.input_norm.weight diff: 0
layers.3.self_attention.query_key_value.weight diff: 0
layers.3.self_attention.dense.weight diff: 0
layers.3.post_attention_norm.weight diff: 0
laye

## Test forward precision

### 对齐记录

(0) 直接跑不对齐的版本，输出基本也OK，top-10的ranking都对

(1) 发现第一个TransformerBlock的输出就有差异，开始排查每一个子模块的输出结果，发现在QK^T的计算时使用的矩阵乘法算法有差异：
* 在Megatron的实现中使用了`torch.baddbmm`，而HuggingFace的版本使用了`torch.matmul`。仿照Megatron的实现方式对HF代码作修改后，一个TransformerBlock的计算结果可以完全对齐。
* 下面也有一个module说明计算结果的差异。

(2) 发现最终logits的输出结果仍然不一致，开始回溯每一个TransformerBlock的计算结果，发现在第3个TransformerBlock就有少量结果无法对齐，逐渐扩散；再细致排查每一个子模块的输出结果，发现第3个TransformerBlock输入侧经过RMSNorm(input_normalization)的输出有一个element有差异。于是先尝试在Megatron代码中把FusedRMSNorm实现改为采用pytorch OP实现的朴素版本，发现差异消失；可以从输入到输出一直保持一致。
* 保持Megatron的所有实现不变，把HF的实现改为FusedMixedRMSNorm，也能一致
* FusedMixedRMSNorm for huggingface generating in multi-devices has bugs, currently worked on one device.

In [16]:
# Since models/llama-megatron-t1p1 == megatron-util-t1p1,
# we use models/llama-megatron-t1p1 to initialize megatron model.
from megatron.arguments import parse_args

inputs_mock = [13, 22, 35]
position_mock = list(range(len(inputs_mock)))
device = 'cuda'

sys.argv = [
    "script.py",
    "--load", os.path.join(this_filepath, "models", "llama-megatron-t1p1"),
    "--micro-batch-size", "1",
    "--no-initialization",
    "--no-async-tensor-model-parallel-allreduce",
    "--train-iters", "10",
    "--no-load-lr-scheduler",
    "--no-load-rng",
    "--position-embedding-type", "rope",
    "--fp16",
    "--attention-softmax-in-fp32",
    "--no-bias-gelu-fusion",
    "--no-bias-dropout-fusion",
    "--no-query-key-layer-scaling",
    "--no-masked-softmax-fusion",
    "--seq-length", str(len(inputs_mock)),
    "--attention-dropout", "0.0",
    "--hidden-dropout", "0.0",
    "--tokenizer-type", "NullTokenizer",
    "--vocab-size", str(32000 - 1),
]
margs = parse_args()

megatron_model, _, _ = Utils.gpt_load_checkpoint(margs, load_scheduler=False)

_set_global_memory_buffer()
model_parallel_cuda_manual_seed(42)

megatron_model.to(device)
megatron_model.eval()

Setting num_layers to 40 from checkpoint
Setting hidden_size to 5120 from checkpoint
Setting ffn_hidden_size to 13824 from checkpoint
Setting num_attention_heads to 40 from checkpoint
Checkpoint did not provide arguments num_query_groups
Setting kv_channels to 128 from checkpoint
Setting max_position_embeddings to 4096 from checkpoint
Setting position_embedding_type to rope from checkpoint
Checkpoint did not provide arguments rotary_percent
Setting add_bias_linear to False from checkpoint
Setting swiglu to True from checkpoint
Setting untie_embeddings_and_output_weights to True from checkpoint
Checkpoint did not provide arguments apply_layernorm_1p
Setting normalization to RMSNorm from checkpoint
Setting padded_vocab_size to 32000 from checkpoint
Setting tensor_model_parallel_size to 1 from checkpoint
Setting pipeline_model_parallel_size to 1 from checkpoint
Checkpoint did not provide arguments virtual_pipeline_model_parallel_size
Checkpoint did not provide arguments num_layers_per_vir

Float16Module(
  (module): GPTModel(
    (language_model): TransformerLanguageModel(
      (embedding): Embedding(
        (word_embeddings): VocabParallelEmbedding()
        (embedding_dropout): Dropout(p=0.0, inplace=False)
      )
      (rotary_pos_emb): RotaryEmbedding()
      (encoder): ParallelTransformer(
        (layers): ModuleList(
          (0-39): 40 x ParallelTransformerLayer(
            (input_norm): MixedFusedRMSNorm()
            (self_attention): ParallelAttention(
              (query_key_value): ColumnParallelLinear()
              (core_attention): CoreAttention(
                (scale_mask_softmax): FusedScaleMaskSoftmax()
                (attention_dropout): Dropout(p=0.0, inplace=False)
              )
              (dense): RowParallelLinear()
            )
            (post_attention_norm): MixedFusedRMSNorm()
            (mlp): ParallelMLP(
              (dense_h_to_4h): ColumnParallelLinear()
              (dense_4h_to_h): RowParallelLinear()
            )
 

In [17]:
input_ids = torch.tensor([inputs_mock]).to(device)
position_ids = torch.tensor([position_mock]).to(device)

seq_length = len(inputs_mock)
att_mask_batch = 1
attention_mask = torch.tril(
    torch.ones((att_mask_batch, seq_length, seq_length), device=device)).view(
        att_mask_batch, 1, seq_length, seq_length)
attention_mask = (attention_mask < 0.5)

megatron_logits = megatron_model.forward(input_ids, position_ids, attention_mask)

=> encoder_input: tensor([[[-3.3112e-03, -5.1117e-04, -1.2665e-03,  ..., -6.7902e-04,
           7.7248e-05, -8.2397e-04]],

        [[-1.0376e-02,  4.6997e-03,  3.4912e-02,  ...,  1.0315e-02,
          -1.2817e-02, -1.1902e-02]],

        [[-2.3842e-06, -6.2585e-06,  6.6757e-06,  ..., -9.1791e-06,
          -3.3379e-06, -1.3113e-06]]], device='cuda:0', dtype=torch.float16,
       grad_fn=<TransposeBackward0>), norm: 1.0498046875
=> Layer-0 hidden_states: tensor([[[-0.0010,  0.0044,  0.0078,  ...,  0.0193,  0.0158,  0.0353]],

        [[-0.0307, -0.0312,  0.0394,  ..., -0.0051, -0.0293, -0.0594]],

        [[ 0.0998,  0.0155,  0.0452,  ...,  0.0493,  0.0040,  0.0309]]],
       device='cuda:0', dtype=torch.float16, grad_fn=<AddBackward0>), norm: 53.34375
=> Layer-1 hidden_states: tensor([[[-0.0136,  0.0066,  0.0076,  ...,  0.0031,  0.0281,  0.0287]],

        [[-0.0829, -0.0149,  0.0231,  ..., -0.0282, -0.0173, -0.0608]],

        [[ 0.0880,  0.0363,  0.0485,  ...,  0.0143,  0.0123,  0.

#### 直接用官方HuggingFace的LLaMA实现 (transformers==4.32.0)

In [18]:
# Run the modeling_llama directly without modification
hf_model = AutoModelForCausalLM.from_pretrained(llama_root,
                                                torch_dtype=torch.float16,
                                                device_map="cuda:0",
                                                trust_remote_code=True).to("cuda")

=> You are using the LlamaForCausalLM with some modifications ...


Loading checkpoint shards: 100%|██████████| 3/3 [00:12<00:00,  4.24s/it]


In [19]:
input_ids = torch.tensor([inputs_mock]).to("cuda")
attention_mask = torch.ones_like(input_ids)
hf_logits = hf_model(
    input_ids=input_ids,
    attention_mask=attention_mask
)

In [20]:
print("megatron_logits:", megatron_logits)
print("hf_logits:", hf_logits.logits)
diff = (megatron_logits != hf_logits.logits).sum()

print(f"final digits >> diff: {diff.item()}")

megatron_logits: tensor([[[-4.8516, -6.6953,  5.3477,  ..., -2.1328, -3.3652, -2.7891],
         [-4.8477, -7.8086,  2.3770,  ..., -2.4609, -2.5469, -3.4395],
         [-1.3828, -6.7852,  0.2759,  ..., -2.1406, -1.5371, -1.1377]]],
       device='cuda:0', grad_fn=<ToCopyBackward0>)
hf_logits: tensor([[[-4.8555, -6.6953,  5.3477,  ..., -2.1328, -3.3652, -2.7891],
         [-4.8477, -7.8125,  2.3750,  ..., -2.4570, -2.5469, -3.4355],
         [-1.3828, -6.8008,  0.2744,  ..., -2.1406, -1.5381, -1.1387]]],
       device='cuda:0', grad_fn=<ToCopyBackward0>)
final digits >> diff: 62666


In [22]:
k = 10
hf_topk = hf_logits[0][0].topk(k, dim=-1)[-1]
megatron_topk = megatron_logits[0].topk(k, dim=-1)[-1]
print(f"top-{k} diff:", (hf_topk != megatron_topk).sum().item())

top-10 diff: 0


#### 在<llama_root>/config.json中开启与Megatron对齐的选项，再初始化新模型
-  use_mlm_rmsnorm: true
-  use_mlm_qkdot: true

In [25]:
# To edit {llama_root}/config.json: 
# -> use_mlm_rmsnorm: true
# -> use_mlm_qkdot: true

hf_model = AutoModelForCausalLM.from_pretrained(llama_root,
                                                torch_dtype=torch.float16,
                                                device_map="cuda:0",
                                                trust_remote_code=True).to("cuda")
input_ids = torch.tensor([inputs_mock]).to("cuda")
attention_mask = torch.ones_like(input_ids)
hf_logits = hf_model(
    input_ids=input_ids,
    attention_mask=attention_mask
)

=> You are using the LlamaForCausalLM with some modifications ...


Loading checkpoint shards: 100%|██████████| 3/3 [00:13<00:00,  4.40s/it]


In [26]:
print("megatron_logits:", megatron_logits)
print("hf_logits:", hf_logits.logits)
diff = (megatron_logits != hf_logits.logits).sum()

print(f"final digits >> diff: {diff.item()}")

megatron_logits: tensor([[[-4.8516, -6.6953,  5.3477,  ..., -2.1328, -3.3652, -2.7891],
         [-4.8477, -7.8086,  2.3770,  ..., -2.4609, -2.5469, -3.4395],
         [-1.3828, -6.7852,  0.2759,  ..., -2.1406, -1.5371, -1.1377]]],
       device='cuda:0', grad_fn=<ToCopyBackward0>)
hf_logits: tensor([[[-4.8516, -6.6953,  5.3477,  ..., -2.1328, -3.3652, -2.7891],
         [-4.8477, -7.8086,  2.3770,  ..., -2.4609, -2.5469, -3.4395],
         [-1.3828, -6.7852,  0.2759,  ..., -2.1406, -1.5371, -1.1377]]],
       device='cuda:0', grad_fn=<ToCopyBackward0>)
final digits >> diff: 0


#### torch.baddbmm in Megatron-LM vs. torch.matmul in Huggingface llama model.

In [40]:
# load inputs
import math
# ! rclone copy oss://inf-alpha/home/megatron/tests/test_convert_ckpt/tensors ./tensors
save_root = f"./tensors"
hf_q_rp = torch.load(os.path.join(save_root, "hf_q_rope.pt"))   # [b, np, sq, hn]
hf_k_rp = torch.load(os.path.join(save_root, "hf_k_rope.pt"))   # [b, np, sk, hn]
mlm_q_rp = torch.load(os.path.join(save_root, "mlm_q_rope.pt")) # [sq, b, np, hn]
mlm_k_rp = torch.load(os.path.join(save_root, "mlm_k_rope.pt")) # [sk, b, np, hn]

print(hf_q_rp.shape, hf_k_rp.shape)
print(mlm_q_rp.shape, mlm_k_rp.shape)

print("mlm_q vs. hf_q:", (mlm_q_rp.transpose(0, 1).transpose(1, 2) != hf_q_rp).sum().item())
print("mlm_k vs. hf_k:", (mlm_k_rp.transpose(0, 1).transpose(1, 2) != hf_k_rp).sum().item())

torch.Size([1, 40, 3, 128]) torch.Size([1, 40, 3, 128])
torch.Size([3, 1, 40, 128]) torch.Size([3, 1, 40, 128])
mlm_q vs. hf_q: 0
mlm_k vs. hf_k: 0


In [54]:
# HF attention_weights
query_states = hf_q_rp 
key_states = hf_k_rp   
head_dim = query_states.size(-1)

attn_weights = torch.matmul(query_states, key_states.transpose(2, 3)) / math.sqrt(head_dim)

print(attn_weights.shape)
print(attn_weights[0][0])  # masks havn't added yet

hf_results = attn_weights[0][0]

torch.Size([1, 40, 3, 3])
tensor([[-0.1736,  0.1776,  0.1196],
        [ 1.6396, -2.5645,  1.4336],
        [ 0.1628, -0.7998,  1.3223]], device='cuda:0', dtype=torch.float16,
       grad_fn=<SelectBackward0>)


In [52]:
# Megatron attention_weights
# [b, np, sq, sk]
output_size = (1,   # query_layer.size(1),
               40,  # query_layer.size(2),
               3,   # query_layer.size(0),
               3,)  # key_layer.size(0))

query_layer = mlm_q_rp
key_layer = mlm_k_rp

# [sq, b, np, hn] -> [sq, b * np, hn]
query_layer = query_layer.reshape(output_size[2],
                                  output_size[0] * output_size[1], -1)
# [sk, b, np, hn] -> [sk, b * np, hn]
key_layer = key_layer.view(output_size[3],
                           output_size[0] * output_size[1], -1)

matmul_input_buffer = torch.empty(
    (output_size[0]*output_size[1], output_size[2], output_size[3]),
    dtype=query_layer.dtype,
    device="cuda")

norm_factor = math.sqrt(query_layer.size(-1))
matmul_result = torch.baddbmm(
            matmul_input_buffer,
            query_layer.transpose(0, 1),   # [b * np, sq, hn]
            key_layer.transpose(0, 1).transpose(1, 2),  # [b * np, hn, sk]
            beta=0.0, alpha=(1.0/norm_factor))

print(matmul_result.shape)
print(matmul_result[0])
mlm_results = matmul_result[0] 

torch.Size([40, 3, 3])
tensor([[-0.1736,  0.1777,  0.1196],
        [ 1.6396, -2.5645,  1.4346],
        [ 0.1627, -0.7998,  1.3223]], device='cuda:0', dtype=torch.float16,
       grad_fn=<SelectBackward0>)


In [51]:
# HF -> Megatron alignment
# Please check the difference results in https://pytorch.org/docs/stable/notes/numerical_accuracy.html#batched-computations-or-slice-computations
# `torch.matmul` is difference to `torch.baddbmm`
#
query_states = hf_q_rp
key_states = hf_k_rp

output_size = (query_states.shape[0],
               query_states.shape[1],
               query_states.shape[2],
               key_states.shape[2],)

query_states = query_states.view(output_size[0] * output_size[1], output_size[2], -1)
key_states = key_states.view(output_size[0] * output_size[1], output_size[3], -1).transpose(-1, -2)

matmul_input_buffer = torch.empty(
    (output_size[0]*output_size[1], output_size[2], output_size[3]),
     dtype=query_layer.dtype,
     device="cuda")

hidden_size_per_attention_head = 5120 // output_size[1]
norm_factor = math.sqrt(hf_q_rp.size(-1))
matmul_result = torch.baddbmm(
            matmul_input_buffer,
            query_states,   # [b * np, sq, hn]
            key_states,  # [b * np, hn, sk]
            beta=0.0, alpha=(1.0/norm_factor))

matmul_result = matmul_result.view(*output_size)
print(matmul_result[0][0])

hf2mlm_results = matmul_result[0][0]

tensor([[-0.1736,  0.1777,  0.1196],
        [ 1.6396, -2.5645,  1.4346],
        [ 0.1627, -0.7998,  1.3223]], device='cuda:0', dtype=torch.float16,
       grad_fn=<SelectBackward0>)


In [60]:
print(f"hf  results:\n\t{torch.tril(hf_results)}\n")
print(f"mlm results:\n\t{torch.tril(mlm_results)}\n")
print(f"hf2mlm results:\n\t{torch.tril(hf2mlm_results)}\n")

hf  results:
	tensor([[-0.1736,  0.0000,  0.0000],
        [ 1.6396, -2.5645,  0.0000],
        [ 0.1628, -0.7998,  1.3223]], device='cuda:0', dtype=torch.float16,
       grad_fn=<TrilBackward0>)

mlm results:
	tensor([[-0.1736,  0.0000,  0.0000],
        [ 1.6396, -2.5645,  0.0000],
        [ 0.1627, -0.7998,  1.3223]], device='cuda:0', dtype=torch.float16,
       grad_fn=<TrilBackward0>)

hf2mlm results:
	tensor([[-0.1736,  0.0000,  0.0000],
        [ 1.6396, -2.5645,  0.0000],
        [ 0.1627, -0.7998,  1.3223]], device='cuda:0', dtype=torch.float16,
       grad_fn=<TrilBackward0>)



#### 其他
- hf的实现中加了debug选项，可以把中间计算结果落盘
- mlm的实现中直接加debug代码会影响正常逻辑，因此将修改的代码也上传到oss上，以后如果再碰到对精度的问题可以快速开始
- 对mlm的代码修改放在oss目录 oss://inf-alpha/home/megatron/tests/test_convert_ckpt/code/Megatron-LM/
- 下面的代码主要是对中间结果进行比较，直接运行不了仅作存档

In [27]:
##
save_root = "<path to tensors>"
idx = 0
hf_tensor = torch.load(os.path.join(save_root, f"hf.{idx}.pt"))
mlm_tensor = torch.load(os.path.join(save_root, f"mlm.{idx}.pt"))

hf_tensors = []
mlm_tensors = []
for i in range(40):
    if os.path.exists(os.path.join(save_root, f"hf.{i}.pt")) and \
       os.path.exists(os.path.join(save_root, f"mlm.{i}.pt")):
        hf_tensors.append(torch.load(os.path.join(save_root, f"hf.{i}.pt")))
        mlm_tensors.append(torch.load(os.path.join(save_root, f"mlm.{i}.pt")))

hf_wte = torch.load(os.path.join(save_root, f"hf.wte.pt"))
mlm_wte = torch.load(os.path.join(save_root, f"mlm.wte.pt"))

hf_in = torch.load(os.path.join(save_root, "hf_input_norm.pt"))
mlm_in = torch.load(os.path.join(save_root, "mlm_input_norm.pt"))

hf_q = torch.load(os.path.join(save_root, "hf_q.pt"))
mlm_q = torch.load(os.path.join(save_root, "mlm_q.pt"))
hf_k = torch.load(os.path.join(save_root, "hf_k.pt"))
mlm_k = torch.load(os.path.join(save_root, "mlm_k.pt"))
hf_q_rp = torch.load(os.path.join(save_root, "hf_q_rope.pt"))
mlm_q_rp = torch.load(os.path.join(save_root, "mlm_q_rope.pt"))
hf_k_rp = torch.load(os.path.join(save_root, "hf_k_rope.pt"))
mlm_k_rp = torch.load(os.path.join(save_root, "mlm_k_rope.pt"))

hf_attn_wi = torch.load(os.path.join(save_root, "hf_attn_wi.pt"))
mlm_attn_wi = torch.load(os.path.join(save_root, "mlm_attn_wi.pt"))

hf_attn_w = torch.load(os.path.join(save_root, "hf_attn_w.pt"))
mlm_attn_w = torch.load(os.path.join(save_root, "mlm_attn_w.pt"))
hf_attn_sm = torch.load(os.path.join(save_root, "hf_attn_sm.pt"))
mlm_attn_sm = torch.load(os.path.join(save_root, "mlm_attn_sm.pt"))

hf_ao = torch.load(os.path.join(save_root, "hf_attn_output.pt"))
mlm_ao = torch.load(os.path.join(save_root, "mlm_attn_output.pt"))

hf_pn = torch.load(os.path.join(save_root, "hf_post_norm.pt"))
mlm_pn = torch.load(os.path.join(save_root, "mlm_post_norm.pt"))

hf_mo = torch.load(os.path.join(save_root, "hf_mlp_out.pt"))
mlm_mo = torch.load(os.path.join(save_root, "mlm_mlp_out.pt"))

hf_fn = torch.load(os.path.join(save_root, "hf.fn.pt"))
mlm_fn = torch.load(os.path.join(save_root, "mlm.fn.pt"))

In [12]:
for i, (_hf_tensor, _mlm_tensor) in enumerate(zip(hf_tensors, mlm_tensors)):
    _hf_tensor = _hf_tensor.squeeze()
    _mlm_tensor = _mlm_tensor.squeeze()
    diff = (_hf_tensor != _mlm_tensor).sum().item()
    print(_hf_tensor)
    print(_mlm_tensor)
    print(f"layer-{i}[{_hf_tensor.shape}]: {diff}")    

tensor([[-0.0010,  0.0044,  0.0078,  ...,  0.0193,  0.0158,  0.0353],
        [-0.0307, -0.0312,  0.0394,  ..., -0.0051, -0.0293, -0.0594],
        [ 0.0998,  0.0155,  0.0452,  ...,  0.0493,  0.0040,  0.0309]],
       device='cuda:0', dtype=torch.float16, grad_fn=<SqueezeBackward0>)
tensor([[-0.0010,  0.0044,  0.0078,  ...,  0.0193,  0.0158,  0.0353],
        [-0.0307, -0.0312,  0.0394,  ..., -0.0051, -0.0293, -0.0594],
        [ 0.0998,  0.0155,  0.0452,  ...,  0.0493,  0.0040,  0.0309]],
       device='cuda:0', dtype=torch.float16, grad_fn=<SqueezeBackward0>)
layer-0[torch.Size([3, 5120])]: 0
tensor([[-0.0136,  0.0066,  0.0076,  ...,  0.0031,  0.0281,  0.0287],
        [-0.0829, -0.0149,  0.0231,  ..., -0.0282, -0.0173, -0.0608],
        [ 0.0880,  0.0363,  0.0485,  ...,  0.0143,  0.0123,  0.0161]],
       device='cuda:0', dtype=torch.float16, grad_fn=<SqueezeBackward0>)
tensor([[-0.0136,  0.0066,  0.0076,  ...,  0.0031,  0.0281,  0.0287],
        [-0.0829, -0.0149,  0.0231,  ..., -0

In [13]:
src = hf_in.squeeze()
dst = mlm_in.squeeze()
print("input_layernorm:", (src != dst).sum().item())

src = mlm_q.squeeze()
dst = hf_q.squeeze().transpose(0, 1)
print("q:", (src != dst).sum())

src = mlm_k.squeeze()
dst = hf_k.squeeze().transpose(0, 1)
print("k:", (src != dst).sum())

src = mlm_q_rp.squeeze()
dst = hf_q_rp.squeeze().transpose(0, 1)
print("q_after_rope:", (src != dst).sum())

src = mlm_k_rp.squeeze()
dst = hf_k_rp.squeeze().transpose(0, 1)
print("k_after_rope:", (src != dst).sum())

src = hf_attn_wi.squeeze()
dst = mlm_attn_wi.squeeze()
# print(src[0], dst[0])
print("attn_wts (because difference mask format):", (src != dst).sum().item())

src = hf_attn_w.squeeze()
dst = mlm_attn_w.squeeze()
print("attn_softmax:", (src != dst).sum().item())

src = hf_attn_sm.squeeze()
dst = mlm_attn_sm.squeeze()
print("attn_outputs:", (src != dst).sum().item())

src = hf_ao.squeeze()
dst = mlm_ao.squeeze()
print("self_attn_outputs:", (src != dst).sum().item())

input_layernorm: 0
q: tensor(0, device='cuda:0')
k: tensor(0, device='cuda:0')
q_after_rope: tensor(0, device='cuda:0')
k_after_rope: tensor(0, device='cuda:0')
attn_wts (because difference mask format): 120
attn_softmax: 0
attn_outputs: 0
self_attn_outputs: 0
